In [18]:
import os
import pandas as pd
import json
import numpy as np

from dotenv import load_dotenv
load_dotenv()

True

In [31]:
import tweepy
import traceback

In [2]:
import spacy
import torch

from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import TfidfVectorizer
from spacymoji import Emoji

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier

from transformers import TFAutoModel, BertTokenizerFast

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import fbeta_score

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [3]:
def get_src_text(row):
    path = "data/en/" + row['event'] + "/" + str(row['threadid']) + "/source-tweets/" + str(row['tweetid']) + ".json"
    with open(path, "r") as f:
        source = json.loads(f.read())
        return source['text']

def is_true(row):
    print(row)
    path = "data/en/" + row['event'] + "/" + str(row['threadid']) + "/annotation.json"
    with open(path, "r") as f:
        source = json.loads(f.read())
        return str(source.get('true', 'unverified'))

In [23]:
# Dataset used: https://figshare.com/articles/dataset/PHEME_rumour_scheme_dataset_journalism_use_case/2068650
df = pd.read_json("data/en-scheme-annotations.json", dtype = {"threadid": str, "tweetid": str}, lines=True)

df['true'] = df.apply(lambda row: is_true(row), axis=1)
df['src_text'] = df.apply(lambda row: get_src_text(row), axis=1)

print(df.head())

event                  putinmissing
threadid         577258317942149120
tweetid          577258317942149120
support                  supporting
evidentiality             url-given
certainty          somewhat-certain
Name: 0, dtype: object
event                  putinmissing
threadid         576755174531862529
tweetid          576755174531862529
support                  supporting
evidentiality             url-given
certainty          somewhat-certain
Name: 1, dtype: object
event                  putinmissing
threadid         576319832800555008
tweetid          576319832800555008
support                  supporting
evidentiality             url-given
certainty          somewhat-certain
Name: 2, dtype: object
event                  putinmissing
threadid         576513463738109954
tweetid          576513463738109954
support                     denying
evidentiality             url-given
certainty                   certain
Name: 3, dtype: object
event                  charliehebdo
threadid

event                      ferguson
threadid         499366666300846081
tweetid          499366666300846081
support                  supporting
evidentiality             url-given
certainty                   certain
Name: 120, dtype: object
event                      ferguson
threadid         500294803402137600
tweetid          500294803402137600
support                  supporting
evidentiality             url-given
certainty          somewhat-certain
Name: 121, dtype: object
event                      ferguson
threadid         500377906305327104
tweetid          500377906305327104
support                  supporting
evidentiality             url-given
certainty          somewhat-certain
Name: 122, dtype: object
event                      ferguson
threadid         500290456845299714
tweetid          500290456845299714
support                  supporting
evidentiality             url-given
certainty                   certain
Name: 123, dtype: object
event                      ferguson


event                  charliehebdo
threadid         552811386259386370
tweetid          552811386259386370
support                  supporting
evidentiality             url-given
certainty          somewhat-certain
Name: 208, dtype: object
event                  charliehebdo
threadid         553587303172833280
tweetid          553587303172833280
support                  supporting
evidentiality             url-given
certainty                   certain
Name: 209, dtype: object
event                prince-toronto
threadid         529689679411810304
tweetid          529689679411810304
support                  supporting
evidentiality           no-evidence
certainty                   certain
Name: 210, dtype: object
event                prince-toronto
threadid         529713467184676864
tweetid          529713467184676864
support                  supporting
evidentiality           no-evidence
certainty                   certain
Name: 211, dtype: object
event                prince-toronto


          event            threadid             tweetid     support  \
0  putinmissing  577258317942149120  577258317942149120  supporting   
1  putinmissing  576755174531862529  576755174531862529  supporting   
2  putinmissing  576319832800555008  576319832800555008  supporting   
3  putinmissing  576513463738109954  576513463738109954     denying   
4  charliehebdo  552783667052167168  552783667052167168  supporting   

  evidentiality         certainty        true  \
0     url-given  somewhat-certain  unverified   
1     url-given  somewhat-certain  unverified   
2     url-given  somewhat-certain  unverified   
3     url-given           certain  unverified   
4     url-given           certain           1   

                                            src_text  
0  Was Vladimir Putin netralized by an internal c...  
1  Coup? RT @jimgeraghty: Rumors all Russian mili...  
2  Hoppla! @L0gg0l: Swiss Rumors: Putin absence d...  
3  Putin reappears on TV amid claims he is unwell...  
4  

In [17]:
train_text, temp_text, train_labels, temp_labels = train_test_split(df['src_text'], df['true'], 
                                                                    random_state=2018, 
                                                                    test_size=0.3)


val_text, test_text, val_labels, test_labels = train_test_split(temp_text, temp_labels, 
                                                                random_state=2018, 
                                                                test_size=0.5, 
                                                                stratify=temp_labels)

train_frame = pd.DataFrame([train_text, train_labels])
val_frame = pd.DataFrame([val_text, val_labels])
test_frame = pd.DataFrame([test_text, test_labels])

In [6]:
print(val_labels.value_counts())
print(train_text)

0             22
1             18
unverified     5
Name: true, dtype: int64
108    PRINCE IN TORONTO TONIGHT:\n@3RDEYEGIRL tweete...
85     Up to 20 held hostage in Sydney Lindt Cafe sie...
170    Five hostages have escaped the besieged Lindt ...
76     #sydneysiege is over. 2 confirmed dead, #PrayF...
286    BREAKING: Three hostages appear to have escape...
                             ...                        
156    BREAKING  @SkyBusiness: freed hostage borne hi...
265    DETAILS: The hostage site is Lindt Chocolat Ca...
226    Currently the #FoxNews website has zero, repea...
102    Police convoy and helicopters are rushing to s...
250    Local media: 3 people appear to escape from Ma...
Name: src_text, Length: 207, dtype: object


In [53]:
nlp_spacymoji = spacy.load("en_core_web_sm")
emoji = Emoji(nlp_spacymoji, merge_spans=True)
nlp_spacymoji.add_pipe('emoji', first=True)
# tokenised_train = train_frame.apply(lambda row: nlp_spacymoji(row['src_text']))
# #tok_train = nlp_spacymoji(train_text)
# print(tokenised_train)

def spacy_tokeniser(text):
    tokens = []
    for w in nlp_spacymoji(text):
        tokens.append(w.lemma_.lower())
    return tokens

In [57]:
# Get features for TF-IDF
tfidf = TfidfVectorizer()#tokenizer=spacy_tokeniser)
tfidf.fit(train_text)
train_features = tfidf.transform(train_text)
validation_features = tfidf.transform(val_text)
test_features = tfidf.transform(test_text)

In [58]:
print(train_features)

  (0, 1199)	0.24445295777678855
  (0, 1130)	0.2633723219567455
  (0, 1117)	0.3973731771501845
  (0, 1114)	0.212110091085962
  (0, 1110)	0.10389814156046775
  (0, 1094)	0.1838272529799555
  (0, 959)	0.2633723219567455
  (0, 950)	0.20491728264962708
  (0, 864)	0.19319072690600506
  (0, 799)	0.2633723219567455
  (0, 787)	0.12850499352435185
  (0, 776)	0.2633723219567455
  (0, 725)	0.22061737780894516
  (0, 584)	0.19319072690600506
  (0, 577)	0.13701228024733503
  (0, 555)	0.08130279123838846
  (0, 207)	0.2633723219567455
  (0, 156)	0.2633723219567455
  (0, 26)	0.2633723219567455
  (1, 1144)	0.25764378654454123
  (1, 1110)	0.15511401535306543
  (1, 1064)	0.18994197665972787
  (1, 995)	0.26280630051960463
  (1, 882)	0.39319989537825556
  (1, 643)	0.28842276607929185
  :	:
  (205, 507)	0.22431768606672714
  (205, 343)	0.297348050812346
  (205, 288)	0.297348050812346
  (205, 262)	0.14337508668613588
  (205, 237)	0.22431768606672714
  (205, 121)	0.17604724059817461
  (205, 110)	0.1583217914753

In [38]:
def evaluation_summary(description, true_labels, predictions):
  print("Evaluation for: " + description)
  print(classification_report(true_labels, predictions,  digits=3, zero_division=0))
  print('\nConfusion matrix:\n',confusion_matrix(true_labels, predictions))

In [61]:
bert = TFAutoModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/536M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [56]:
# SVC
svc = SVC(kernel='rbf')
svc_model = svc.fit(train_features, train_labels)
print(svc_model.score(validation_features, val_labels))
svc_predicted_labels = svc_model.predict(validation_features)

evaluation_summary("SVC", val_labels, svc_predicted_labels)
svc_test = svc_model.predict(test_features)
evaluation_summary("SVC test", test_labels, svc_test)

# Logistic Regression with TF-IDF
lr_tfidf = LogisticRegression()
lr_model_tfidf = lr_tfidf.fit(train_features, train_labels)
print(lr_model_tfidf.score(validation_features, val_labels))
print(lr_model_tfidf.score(test_features, test_labels))

lr_predicted_labels_tfidf = lr_model_tfidf.predict(validation_features)
evaluation_summary("LR (TF-IDF)", val_labels, lr_predicted_labels_tfidf)
lrtfidf_test = lr_model_tfidf.predict(test_features)
evaluation_summary("LR (TF-IDF) test", test_labels, lrtfidf_test)

# Dummy Majority
dumb = DummyClassifier(strategy='most_frequent')
dumb.fit(train_features, train_labels)
print(dumb.score(validation_features, val_labels))
dumb_validation_predicted_labels = dumb.predict(validation_features)
evaluation_summary("Dummy majority", val_labels, dumb_validation_predicted_labels)
dumb_test = dumb.predict(test_features)
evaluation_summary("Dummy MF test", test_labels, dumb_test)

0.6222222222222222
Evaluation for: SVC
              precision    recall  f1-score   support

           0      0.737     0.636     0.683        22
           1      0.538     0.778     0.636        18
  unverified      0.000     0.000     0.000         5

    accuracy                          0.622        45
   macro avg      0.425     0.471     0.440        45
weighted avg      0.576     0.622     0.588        45


Confusion matrix:
 [[14  8  0]
 [ 4 14  0]
 [ 1  4  0]]
Evaluation for: SVC test
              precision    recall  f1-score   support

           0      0.833     0.455     0.588        22
           1      0.500     0.889     0.640        18
  unverified      1.000     0.200     0.333         5

    accuracy                          0.600        45
   macro avg      0.778     0.514     0.521        45
weighted avg      0.719     0.600     0.581        45


Confusion matrix:
 [[10 12  0]
 [ 2 16  0]
 [ 0  4  1]]
0.6222222222222222
0.6
Evaluation for: LR (TF-IDF)
         

In [21]:
def get_src_text_by_index(row):
    return df.iloc[row.index]['src_text']

accuracy = pd.DataFrame([test_labels, lrtfidf_test])
print(accuracy)
print(lrtfidf_test)
print(test_labels)
print(type(test_labels))
test_df = pd.DataFrame(test_labels)
test_df['predicted'] = lrtfidf_test
print(test_df.apply(lambda row: get_src_text_by_index(row)))
print(test_df)
#test_df = pd.DataFrame({"tweet_id":test_labels[0], "actual_label": test_labels[1]})
test_labels.add(lrtfidf_test)
print(test_labels)

           127 18   143  161 32   131  177         19          29   172  ...  \
true         1   1    1    1   0    0    0           0  unverified    0  ...   
Unnamed 0  NaN   0  NaN  NaN   0  NaN  NaN  unverified           1  NaN  ...   

           288  202  162 5    58   69  17          210  223  54   
true         1    0    1   1    1    0   0  unverified    0    0  
Unnamed 0  NaN  NaN  NaN   1  NaN  NaN   0         NaN  NaN  NaN  

[2 rows x 45 columns]
['1' '0' '0' '1' '0' '1' '1' '0' '1' '1' '1' '1' '0' '0' '0' '1' '0' '0'
 '0' 'unverified' '1' '1' '0' '0' '1' '1' '1' '1' '1' '1' '1' '1' '0' '0'
 '1' '1' '0' '1' '1' '1' '0' '1' '1' '0' '1']
127             1
18              1
143             1
161             1
32              0
131             0
177             0
19              0
29     unverified
172             0
92              1
51              1
215             0
7               1
206             0
285             1
269             1
118             0
36              0


In [22]:
# Get output

input_text = "Watch video showing gunfire inside Canada's pa"
input_df = pd.DataFrame({"src_text": [input_text]})
input_features = tfidf.transform(input_df)

predicted_label = lr_model_tfidf.predict(input_features)

print(predicted_label)

['0']


In [ ]:
# BERT

In [5]:
import transformers as ppb
model_class, tokenizer_class, bert_model_name = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

tokenizer = tokenizer_class.from_pretrained(bert_model_name)
model = model_class.from_pretrained(bert_model_name)

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

C:\ProgramData\Anaconda3\lib\site-packages\huggingface_hub\file_download.py:123: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\karli\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.

In [6]:
tokenized = df['src_text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [7]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [8]:
input_ids = torch.tensor(np.array(padded))

with torch.no_grad():
    last_hidden_states = model(input_ids)

In [9]:
features = last_hidden_states[0][:,0,:].numpy()

In [10]:
labels = df['true']
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

In [11]:
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [12]:
lr_clf.score(test_features, test_labels)

0.5866666666666667

In [ ]:
# Cleaning dataset

In [14]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
import re
from nltk.stem.porter import PorterStemmer
from collections import Counter

ps = PorterStemmer()
wnl = nltk.stem.WordNetLemmatizer()

stop_words = stopwords.words('english')
stopwords_dict = Counter(stop_words)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\karli\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\karli\AppData\Roaming\nltk_data...


In [24]:
remove_col = ['threadid','tweetid','support', 'evidentiality', 'certainty', 'event']

# Impute null values with None
def null_process(feature_df):
    col = 'src_text'
    feature_df.loc[feature_df[col].isnull(), col] = "None"
    return feature_df

# Removed unused clumns
def remove_unused_col(df,column_n=remove_col):
    df = df.drop(column_n,axis=1)
    return df

def clean_dataset(df):
    # remove unused column
    df = remove_unused_col(df)
    #impute null values
    df = null_process(df)
    return df

# Cleaning text from unused characters
def clean_text(text):
    text = str(text).replace(r'http[\w:/\.]+', ' ')  # removing urls
    text = str(text).replace(r'[^\.\w\s]', ' ')  # remove everything but characters and punctuation
    text = str(text).replace('[^a-zA-Z]', ' ')
    text = str(text).replace(r'\s\s+', ' ')
    text = text.lower().strip()
    #text = ' '.join(text)    
    return text

## Nltk Preprocessing includes:
# Stop words, Stemming and Lemmetization
def nltk_preprocess(text):
    text = clean_text(text)
    wordlist = re.sub(r'[^\w\s]', '', text).split()
    text = ' '.join([word for word in wordlist if word not in stopwords_dict])
    text = [ps.stem(word) for word in wordlist if not word in stopwords_dict]
    text = ' '.join([wnl.lemmatize(word) for word in wordlist if word not in stopwords_dict])
    return  text

In [25]:
df = clean_dataset(df)
df["src_text"] = df.src_text.apply(nltk_preprocess)
df.head()

,true,src_text
0,unverified,vladimir putin netralized internal coup maybe ...
1,unverified,coup rt jimgeraghty rumor russian military att...
2,unverified,hoppla l0gg0l swiss rumor putin absence due gi...
3,unverified,putin reappears tv amid claim unwell threat co...
4,1,france 10 people dead shooting hq satirical we...


In [ ]:
# New dataset - FakeNewsNet

In [7]:
politifact_fake_df = pd.read_csv("data/FakeNewsNet-master/dataset/politifact_fake.csv")

pf_fake_tweet_ids = politifact_fake_df['tweet_ids'].str.split('\t').explode().tolist()
print(len(pf_fake_tweet_ids))
print()

politifact_real_df = pd.read_csv("data/FakeNewsNet-master/dataset/politifact_real.csv")
pf_real_tweet_ids = politifact_real_df['tweet_ids'].str.split('\t').explode().tolist()
print(len(pf_real_tweet_ids))

# def concat_tweet_ids(row):
#     global pf_fake_tweet_ids
#     print('tweet_ids' in row)
#     if 'tweet_ids' in row and row['tweet_ids'] != "":
#         pf_fake_tweet_ids = pf_fake_tweet_ids + row['tweet_ids'].split("\t")
    
# politifact_fake_df.apply(lambda x: concat_tweet_ids(x))
# print(pf_fake_tweet_ids)

fake_short = pf_fake_tweet_ids[:20]

print(fake_short)

165392

418379
['937349434668498944', '937379378006282240', '937380068590055425', '937384406511005696', '937387493451862016', '937400766024896512', '937406789686980608', '937411332240011266', '937415066810503168', '937427631661768704', '937429898670600192', '937436145004302337', '937438119468699648', '937449906352152576', '937450317142286336', '937451599320027136', '937452013939494912', '937452151227510784', '937453119478423553', '937462176293437443']


In [34]:
# https://stackoverflow.com/questions/72505232/feature-extraction-with-tweet-ids

api_key = os.environ.get('APIKey')
api_key_secret = os.environ.get('APIKeySecret')
access_token = os.environ.get('AccessToken')
access_token_secret = os.environ.get('AccessTokenSecret')
bearer_token = os.environ.get('BearerToken')

api = tweepy.Client(consumer_key=api_key, 
                       consumer_secret=api_key_secret,
                       access_token=access_token, 
                       access_token_secret=access_token_secret,
                       bearer_token=bearer_token, 
                       wait_on_rate_limit=True,
                        )

auth = tweepy.OAuth1UserHandler(
   api_key, api_key_secret, access_token, access_token_secret
)

api = tweepy.API(auth)


IDs = fake_short

tweets_df = pd.DataFrame()

for i in IDs:
    try:
        info_tweet = api.get_status(i, tweet_mode="extended")  
    except Exception as e:
        traceback.print_exc()
        pass
    
    tweets_df = tweets_df.append(pd.DataFrame({'ID': info_tweet.id,
                                             'Text': info_tweet.full_text,
                                             'Created at': info_tweet.created_at,
                                             'User location': info_tweet.user.location,
                                             'Num Followers': info_tweet.user.followers_count,
                                             'Num Friends': info_tweet.user.friends_count,
                                             'Num Favourites': info_tweet.user.favourites_count,
                                             'User description': info_tweet.user.description,
                                             'User verified': info_tweet.user.verified,
                                             'Language': info_tweet.lang}, index=[0]))
    tweets_df = tweets_df.reset_index(drop=True)

tweets_df

Traceback (most recent call last):
  File "<ipython-input-34-ffe512ac46fe>", line 30, in <module>
    info_tweet = api.get_status(i, tweet_mode="extended")
  File "C:\ProgramData\Anaconda3\lib\site-packages\tweepy\api.py", line 46, in wrapper
    return method(*args, **kwargs)
  File "C:\ProgramData\Anaconda3\lib\site-packages\tweepy\api.py", line 902, in get_status
    return self.request(
  File "C:\ProgramData\Anaconda3\lib\site-packages\tweepy\api.py", line 265, in request
    raise Forbidden(resp)
tweepy.errors.Forbidden: 403 Forbidden
453 - You currently have Essential access which includes access to Twitter API v2 endpoints only. If you need access to this endpoint, you’ll need to apply for Elevated access via the Developer Portal. You can learn more here: https://developer.twitter.com/en/docs/twitter-api/getting-started/about-twitter-api#v2-access-leve


NameError: name 'info_tweet' is not defined

In [16]:
tweets_df2.to_csv("final_short.csv")

hello
